In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from dataloader import MRIDataset
from residual3dunet.model import ResidualUNet3D
import matplotlib.pyplot as plt
from utils import get_loaders
import torchvision.transforms.functional as F
import torchvision.transforms as T
import random
import h5py
import numpy as np
from ipywidgets import interact
from elastic_transform import RandomElastic
# import nibabel as nib
#from residual3dunet.modelorig import ResidualUNet3D

import matplotlib.pyplot as plt

In [2]:
dataset = MRIDataset(train=True)
train, val = torch.utils.data.random_split(dataset, [40, 10])

dataloader = DataLoader(dataset = train, batch_size= 1, shuffle= True, num_workers=2)

In [ ]:
# dataset = MRIDataset(train=True, transform=True)

train_kwargs = {'batch_size': 10}
cuda_kwargs = {'num_workers': 1, 'pin_memory': True,'shuffle': True}
train_kwargs.update(cuda_kwargs)

#train, val = random_split(dataset, [40, 10])
# dataloader = DataLoader(dataset = val, **train_kwargs)

In [ ]:
dataloader, valloader = get_loaders(train=True, transform=True, **train_kwargs)

In [3]:
dataiter = iter(dataloader)
data = dataiter.next()
features, labels = data
print(features.shape)
print(labels.shape)

torch.Size([1, 1, 14, 240, 240])
torch.Size([1, 1, 14, 240, 240])


In [4]:
model = ResidualUNet3D(in_channels=1, out_channels=1)

In [5]:
features, labels = features.float().to('cpu'), labels.float().to('cpu')
output = model(features)

torch.Size([1, 32, 14, 240, 240])
torch.Size([1, 64, 14, 120, 120])
torch.Size([1, 128, 14, 60, 60])
torch.Size([1, 256, 14, 30, 30])
torch.Size([1, 512, 14, 15, 15])
torch.Size([1, 256, 14, 30, 30])
torch.Size([1, 128, 14, 60, 60])
torch.Size([1, 64, 14, 120, 120])
torch.Size([1, 32, 14, 240, 240])
torch.Size([1, 1, 14, 240, 240])


In [ ]:
dataset2 = MRIDataset(train=True, transform=T.Compose([
    T.ToTensor(), 
    T.RandomHorizontalFlip(), 
    T.RandomCrop((240,240), padding=50, pad_if_needed=True),
    ]), elastic=True)


In [ ]:

test_loader = DataLoader(dataset = dataset2, batch_size=50, shuffle=False)

# dataiter = iter(test_loader)

# first = next(dataiter)
# second = next(dataiter)

# features1, labels1 = first
# features2, labels2 = second

for data, target in test_loader:
    print(data.shape)
    print(target.shape)



In [ ]:
h5ftrain = h5py.File('dataset/T2train.h5','r')
h5ftrainmask = h5py.File('dataset/T2trainmask.h5','r')

data = h5ftrain[f'T2data_1'][:]
target = h5ftrainmask[f'T2maskdata_1'][:]

print(data.shape)
print(target.shape)

preprocess = RandomElastic(alpha=0, sigma=0.06)
data1, target1 = preprocess(data, target)

print(data1.shape)
print(target1.shape)


In [ ]:
# index = random.randint(0,49)
def explore_3d_image(layer):

    plt.figure(figsize=(5,10))
    plt.imshow(data[1,0,layer,:,:],cmap='gray')
    plt.imshow(target[1,0,layer,:,:],cmap='gray', alpha=0.3)
    plt.title('Explore Layers of Kidney MRI')
    plt.axis('off')
    return layer

interact(explore_3d_image, layer=(0,13))

In [ ]:
def dice_coefficient(pred, target):

    dice_score = 0
    dice_score += (2 * (pred * target).sum()) / (pred + target).sum()

    return dice_score


def iou(pred, target):
    
    pred = pred.int()
    target = target.int()

    intersection = (pred & target).float().sum()
    union = (pred | target).float().sum()   

    iou = (intersection + 1e-6) / (union + 1e-6) 

    return iou.mean()

In [ ]:
num_correct = 0
num_pixels = 0
dice_score = 0

a = torch.zeros((1,5,5))
b = torch.ones((1,5,5))

a[0:3,2:5] = 1
b[2:5,0:2] = 0
b[0:1,2:4] = 0
print(a)
print(b)

num_correct += (a == b).sum()
num_pixels += torch.numel(a)

print(dice_coefficient(a, b))
print(num_correct/num_pixels)
print(iou(a,b))